In [130]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from matplotlib import pyplot as plt

In [131]:
dataset = 'dataset'

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root=dataset, train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root=dataset, train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

### Import Data ###

In [132]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        # there are 2 max poolings which each half the sqrt dim of x.dim
        size_of_flattened = int(64*(x_dim/((2*2)**2)))
        self.encoderLayers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(in_features=size_of_flattened, out_features=256),
            nn.ReLU()
        )

        self.mu_sample_layer = nn.Linear(in_features=256, out_features=z_dim)
        self.log_var_sample_layer = nn.Linear(in_features=256, out_features=z_dim)

        # decoder part
        self.decoder_conv = nn.Sequential(
            nn.Linear(z_dim, 28*28),
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(1, 28, 28)),
            # nn.ConvTranspose2d(in_channels=1, out_channels=8, kernel_size=3, stride=2, output_padding=0),
            # nn.BatchNorm2d(8),
            # nn.ReLU(True),
            # nn.ConvTranspose2d(in_channels = 8, out_channels = 8, kernel_size=3, stride=2, padding=1, output_padding=1),
            # nn.BatchNorm2d(8),
            # nn.ReLU(True),
            # nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=3, stride=2, padding=1, output_padding=1)
            nn.Linear(in_features=28, out_features=28),
            nn.ReLU(),
            nn.Linear(in_features=28, out_features=28),
            nn.ReLU()
        )
        
    def encoder(self, x):
        h = self.encoderLayers(x)
        return self.mu_sample_layer(h), self.log_var_sample_layer(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = self.decoder_conv(z)
        return F.sigmoid(h)  
    
    def forward(self, x):
        # flatten
        mu, log_var = self.encoder(x)
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [ ]:
vae

VAE(
  (encoderLayers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=3136, out_features=256, bias=True)
    (8): ReLU()
  )
  (mu_sample_layer): Linear(in_features=256, out_features=2, bias=True)
  (log_var_sample_layer): Linear(in_features=256, out_features=2, bias=True)
  (decoder_conv): Sequential(
    (0): Linear(in_features=2, out_features=784, bias=True)
    (1): ReLU()
    (2): Unflatten(dim=1, unflattened_size=(1, 28, 28))
    (3): Linear(in_features=28, out_features=28, bias=True)
    (4): ReLU()
    (5): Linear(in_features=28, out_features=28, bias=True)
    (6): ReLU()
  )
)

In [125]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    #print the shapes
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

### Encoder and Decoder ###

In [126]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data
        optimizer.zero_grad()
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [127]:

def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [128]:
#Try running one sample through the autoencoder
sample = next(iter(test_loader))[0][0]
print(sample.unsqueeze(0).shape)
encoderResults = vae.encoder(sample.unsqueeze(0))

sampleResults = vae.sampling(encoderResults[0], encoderResults[1])

decoderResults = vae.decoder(sampleResults)
print(decoderResults.shape)

torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])


In [133]:
for epoch in range(1, 15):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 559.895898
Train Epoch: 1 [10000/60000 (17%)]	Loss: 560.394258
Train Epoch: 1 [20000/60000 (33%)]	Loss: 560.597617
Train Epoch: 1 [30000/60000 (50%)]	Loss: 560.289609
Train Epoch: 1 [40000/60000 (67%)]	Loss: 559.980742
Train Epoch: 1 [50000/60000 (83%)]	Loss: 559.765898
====> Epoch: 1 Average loss: 560.2971
====> Test set loss: 560.2769
Train Epoch: 2 [0/60000 (0%)]	Loss: 560.298242
Train Epoch: 2 [10000/60000 (17%)]	Loss: 560.001602
Train Epoch: 2 [20000/60000 (33%)]	Loss: 560.408789
Train Epoch: 2 [30000/60000 (50%)]	Loss: 560.297852
Train Epoch: 2 [40000/60000 (67%)]	Loss: 560.199766
Train Epoch: 2 [50000/60000 (83%)]	Loss: 560.390625
====> Epoch: 2 Average loss: 560.2953
====> Test set loss: 560.2573
Train Epoch: 3 [0/60000 (0%)]	Loss: 560.357969
Train Epoch: 3 [10000/60000 (17%)]	Loss: 561.169922
Train Epoch: 3 [20000/60000 (33%)]	Loss: 560.628555
Train Epoch: 3 [30000/60000 (50%)]	Loss: 560.375937
Train Epoch: 3 [40000/60000 (67%)]	Loss: 560.31

KeyboardInterrupt: 

### Train ###

In [ ]:
with torch.no_grad():
    z = torch.randn(64, 2)
    sample = vae.decoder(z)
    
    for (i, img) in enumerate(sample):
        plt.subplot(8, 8, i+1)
        plt.imshow(img.view(28, 28).numpy())
        plt.axis('off')
